In [ ]:
from aiida import load_profile
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query
import aiidalab_widgets_base as awb

# Title

In [ ]:
sq = sens_query.SearchSens()
sq